In [3]:
%%capture
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,LSTM, Conv1D, Activation, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.metrics import AUC
import keras


from sz_utils import data_handler
import pandas as pd

import getpass
import os

In [4]:
# check if gpu is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
%%capture
# collect the data
preictal, interictal = data_handler.make_patient_windows("chb01")

# make the labels
X = np.concatenate((preictal, interictal), axis=0)
y = np.concatenate((np.ones((preictal.shape[0], 1)), np.zeros((interictal.shape[0], 1))), axis=0)

# Shuffle the data
shuffle_indices = np.random.permutation(np.arange(X.shape[0]))
X = X[shuffle_indices]
y = y[shuffle_indices]

# Split the data into train and test
train_size = int(X.shape[0] * 0.8)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

# Split test data into validation and test
val_size = int(X_test.shape[0] * 0.5)
X_val = X_test[:val_size]
y_val = y_test[:val_size]
X_test = X_test[val_size:]
y_test = y_test[val_size:]

In [8]:
# shapes
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape, "X_val shape:", X_val.shape, "y_val shape:", y_val.shape, "X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (672, 1280, 22) y_train shape: (672, 1) X_val shape: (84, 1280, 22) y_val shape: (84, 1) X_test shape: (84, 1280, 22) y_test shape: (84, 1)


In [9]:
num_classes = 2 
input_shape_dataset = (X_train.shape[1], X_train.shape[2])
input_shape_dataset

def create_model_cnn_basic_1_layer(
    input_shape_dataset: tuple = input_shape_dataset,
    num_classes: int = num_classes,
    debug: bool = False,
    filters: int = 256,
    kernel_size: int = 3,
    pool_size: int = 2,
    dropout: float = 0.1,
    dense_size: int = 64,
    loss: str = "binary_crossentropy",
    optimizer: str = "adam",
    metrics: list = ["accuracy"],

) -> tf.keras.Model:

    """This function creates a basic convolutional neural network model with 2 convolutional layers, 2 dense layers and a softmax layer

    :param input_shape_dataset: shape of the input data
    :type input_shape_dataset: tuple
    :param num_classes: number of classes
    :type num_classes: int
    :return: return a model
    :rtype: tf.keras.Model
    """

    if debug:
        print("------------model summary---------------")
        print("input_shape_dataset", input_shape_dataset)
        print("num_classes", num_classes)

    input_shape_dataset: tuple
    
    model = Sequential()

    model.add(Conv1D(filters, kernel_size, input_shape=(input_shape_dataset)))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    return model

In [10]:
dataset_ns = (X, y)
dataset_ns[0].shape 

(840, 1280, 22)

In [36]:
import matplotlib.pyplot as plt
from dataclasses import dataclass
import tensorflow as tf
import mlflow
from sklearn.model_selection import KFold


class Experiment:
    experiment_name: str
    model_name: str
    model: tf.keras.Model
    dataset: tuple
    hyperparameters: dict
    metrics: dict

    def __init__(self, experiment_name, model_name, model, dataset, hyperparameters, metrics, num_folds=5):
        self.experiment_name = experiment_name
        self.model_name = model_name
        self.model = model
        self.dataset = dataset
        self.hyperparameters = hyperparameters
        self.metrics = metrics
        self.num_folds = num_folds
        self.fold_no = 1
        self.acc_per_fold = []
        self.loss_per_fold = []
        self.kfold = KFold(n_splits=self.num_folds, shuffle=True)
        self.train = None
        self.test = None

    def fit_evaluate(self):
        try:
            mlflow.tensorflow.autolog()
            for i, (train, test) in enumerate(self.kfold.split(self.dataset[0], self.dataset[1])):
                with mlflow.start_run(run_name=f"fold_{i}", nested=True):
                    self.train = train
                    self.test = test
                    history = self.model.fit(self.dataset[0][train], self.dataset[1][train], epochs=self.hyperparameters["epochs"], verbose=1)
                    scores = self.model.evaluate(self.dataset[0][test], self.dataset[1][test], verbose=0)
                    print(f'Score for fold {self.fold_no}: {self.model.metrics_names[0]} of {scores[0]}; {self.model.metrics_names[1]} of {scores[1]*100}%')
                    self.acc_per_fold.append(scores[1] * 100)
                    mlflow.log_param("acc_per_fold", self.acc_per_fold)
                    self.loss_per_fold.append(scores[0])
                    self.log_metrics(history)
                    self.fold_no += 1
                # mlflow.end_run()

            mlflow.log_artifact(self._log_graphs(history), artifact_path='Artifacts')

            return history
        except Exception as e:
            print("El error se produce en el fit, el error es: ", e)

    def set_experiment(self):
        mlflow.set_experiment(self.experiment_name)

    def log_params(self):
        mlflow.log_param("model_name", self.model_name)
        for key, value in self.hyperparameters.items():
            mlflow.log_param(key, value)

    def log_metrics(self, history):
        for metric_name, metric_values in history.history.items():
            for epoch, value in enumerate(metric_values):
                mlflow.log_metric(f"{metric_name}", value, step=epoch)

    def _log_graphs(self, history):
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
            
            # Accuracy plot
            ax[0].plot(history.history['accuracy'], label='train')
            ax[0].plot(history.history['val_accuracy'], label='val')
            ax[0].set_title('Accuracy')
            ax[0].set_xlabel('Epoch')
            ax[0].set_ylabel('Accuracy')
            ax[0].legend()
        
            # Loss plot
            ax[1].plot(history.history['loss'], label='train')
            ax[1].plot(history.history['val_loss'], label='val')
            ax[1].set_title('Loss')
            ax[1].set_xlabel('Epoch')
            ax[1].set_ylabel('Loss')
            ax[1].legend()
            
            # Save the plots to a file
            plt.tight_layout()
            plt.savefig('graphs.png')
            return 'graphs.png'
            
    def log_artifacts(self):
        pass

    def run(self):
        self.set_experiment()
        self.log_params()
        history = self.fit_evaluate()
        artifact_path = self._log_graphs(history)
        mlflow.log_artifact(artifact_path, artifact_path='Artifacts')


In [37]:
# TODO: Analyze if it is necessary to use gc.collect()
# import gc
# gc.collect()

def register_experiment(experiment_name, model_name, model, dataset, hyperparameters, metrics, num_folds=5):
    # create an instance of Experiment class
    experiment = Experiment(experiment_name, model_name, model, dataset, hyperparameters, metrics, num_folds=num_folds)

    # execute the methods of Experiment class
    with mlflow.start_run(nested=True):
        experiment.set_experiment()
        experiment.log_params()
        experiment.fit_evaluate()
        experiment.log_artifacts()
        experiment.log_metrics()

        return experiment   




In [38]:
experiment_1 = Experiment(
    experiment_name = "CNN_autolog",
    model_name = "CNN_basic_1_layer",
    model = create_model_cnn_basic_1_layer(),
    dataset = dataset_ns,
    hyperparameters = {
        "epochs": 11,
        "filters": 256,
        "kernel_size": 3,
        "pool_size": 1,
        "dropout": 0.1,
        "dense_size": 32,
        "loss": "binary_crossentropy",
        "optimizer": "adam",
        
    },metrics = ["accuracy"],
)

experiment_2 = Experiment(
    experiment_name = "CNN_autolog",
    model_name = "CNN_basic_1_layer",
    model = create_model_cnn_basic_1_layer(),
    dataset = dataset_ns,
    hyperparameters = {
        "epochs": 17,
        "filters": 128,
        "kernel_size": 3,
        "pool_size": 2,
        "dropout": 0.1,
        "dense_size": 16,
        "loss": "binary_crossentropy",
        "optimizer": "adam",
        
    },metrics = ["accuracy"],
)

In [39]:
experiment_1

In [40]:
register_experiment(experiment_1)

TypeError: register_experiment() missing 5 required positional arguments: 'model_name', 'model', 'dataset', 'hyperparameters', and 'metrics'

In [30]:
experiments = [experiment_1, experiment_2]

for experiment in experiments:
    register_experiment(experiment)
    

TypeError: register_experiment() missing 5 required positional arguments: 'model_name', 'model', 'dataset', 'hyperparameters', and 'metrics'